In [1]:
from numba import cuda
import cupy as cp
import numpy as np

In [2]:
@cuda.jit
def cuda_prime(C, start):
    i, j = cuda.grid(2)  
    if i < C.shape[0] and j < C.shape[1]:
        check = 0
        for k in range(2, i*C.shape[1] + j + start):
            if(k*k>i*C.shape[1] + j + start):
                break
            if((i*C.shape[1] + j + start)% k == 0):
                check = 1
                break
        if(check == 0):
            C[i, j] = i*C.shape[1] + j + start

In [3]:
def cpu_prime(start, size):
    sum = 0
    for i in range(start, size + start):
        check = 0
        for j in range(start, int((size + start)**0.5)):
            if(i%j==0):
                check = 1
                break
        if(check == 0):
            sum = sum + 1

In [4]:
cp.random.seed(42)
sizeMatrix = 800
start = 2
C = cp.zeros((sizeMatrix,sizeMatrix), dtype=np.int32)
TPB = 16
threadsperblock = (TPB, TPB)  # each block will contain 16x16 threads, typically 128 - 512 threads/block
blockspergrid_x = int(np.ceil(sizeMatrix / threadsperblock[0]))
blockspergrid_y = int(np.ceil(sizeMatrix / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)
print(blockspergrid)
print(f"The kernel will be executed up to element {threadsperblock[0]*blockspergrid_x}")

(50, 50)
The kernel will be executed up to element 800


In [5]:
%%time
for i in range (100):
    cuda_prime[blockspergrid, threadsperblock](C, start)

Wall time: 56.7 s


In [6]:
%%time
cpu_prime(start, sizeMatrix*sizeMatrix)

Wall time: 6.96 s
